In [1]:
import pandas as pd

kospi_ratio = pd.read_csv('../data/processed_data/재무비율.csv')
kospi_ind = pd.read_excel('../data/raw_data/코스피_개별재무비율.xlsx')
kosdaq_ind = pd.read_excel('../data/raw_data/코스닥_개별재무비율.xlsx')

In [2]:
listed = pd.concat([kospi_ind, kosdaq_ind], ignore_index=True)

kospi_ratio.iloc[:, 9:-1]

,총자본증가율(IFRS연결),유형자산증가율(IFRS연결),비유동생물자산증가율(IFRS연결),투자부동산증가율(IFRS연결),비유동자산증가율(IFRS연결),유동자산증가율(IFRS연결),재고자산증가율(IFRS연결),자기자본증가율(IFRS연결),매출액증가율(IFRS연결),정상영업이익증가율(IFRS연결),...,인건비(IFRS연결)(백만원),금융비용(IFRS연결)(백만원),임차료(IFRS연결)(백만원),세금과공과(IFRS연결)(백만원),감가상각비(IFRS연결)(백만원),종업원1인당 부가가치(IFRS연결)(백만원).1,총자본투자효율(IFRS연결).1,기계투자효율(IFRS연결).1,부가가치율(IFRS연결).1,종업원수(IFRS연결)
0,-4.56,-11.48,0.0,-1.61,-7.14,0.15,19.60,14.56,-7.80,-63.29,...,46876.84,3589.40,0.0,0.0,14203.75,NaN,19.72,98.91,29.06,NaN
1,-12.91,-11.57,0.0,-1.64,-11.10,-15.98,-34.15,-37.30,-10.33,-54.71,...,0.00,0.00,0.0,0.0,0.00,NaN,-3.89,0.00,-5.57,NaN
2,-17.07,-12.80,0.0,-1.66,-15.39,-20.10,-14.41,-36.34,-19.06,-177.01,...,39598.55,3990.88,0.0,0.0,11694.91,NaN,8.59,44.66,12.60,NaN
3,4.13,-9.82,0.0,-1.23,-3.95,8.39,40.49,0.28,17.15,4.17,...,802635.00,94086.89,124047.0,54145.0,51212.00,NaN,13.73,2422.60,16.47,NaN
4,2.67,0.05,0.0,0.26,0.98,3.46,3.91,-22.58,6.79,0.00,...,913926.00,86015.38,123239.0,62921.0,59003.00,NaN,4.00,731.03,4.62,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,30291.89,3425.26,0.0,0.0,8607.60,NaN,20.66,165.10,32.15,NaN
390,-12.56,-17.12,0.0,735.67,-6.64,-23.82,-60.05,-4.41,19.60,2124.67,...,28696.72,2408.62,0.0,0.0,7715.50,NaN,27.08,192.48,30.81,NaN


---
### 전체에 대해서 값이 없는 행(기업) 탐색

In [3]:
# 모든 컬럼에서 값이 없는(결측치) 행을 찾기
empty_rows = kospi_ratio[kospi_ratio.iloc[:, 9:-1].isnull().all(axis=1)]

print(len(empty_rows.drop_duplicates('회사명')['회사명']))
print('------------------------------------------')
print(empty_rows.drop_duplicates('회사명')['회사명'])

35
------------------------------------------
7          (주)대호에이엘
12       (주)신화다이나믹스
74          백광산업(주)
78      삼성바이오로직스(주)
95        코스모신소재(주)
98         코스모화학(주)
105       한국전력기술(주)
114      한솔아트원제지(주)
131           (주)광무
144          (주)뉴보텍
156        (주)디지털대성
158          (주)메지온
165           (주)삼일
167           (주)상보
173         (주)서울제약
195          (주)시큐브
211           (주)씨젠
227        (주)에스에스알
248         (주)와이어블
276      (주)제이테크놀로지
288     (주)케이바이오컴퍼니
293    (주)케이에스인더스트리
298     (주)케이지모빌리언스
300      (주)케이지이니시스
308        (주)타이거일렉
311       (주)티에스넥스젠
330        (주)한진피앤씨
336         경남제약(주)
341       대한그린파워(주)
343          더라미(주)
349         디에스티(주)
352        삼영이엔씨(주)
358        씨앤티85(주)
372       우양에이치씨(주)
383       한솔아이원스(주)
Name: 회사명, dtype: object


In [4]:
print(empty_rows.shape)
print(empty_rows.drop_duplicates('회사명').shape)

(89, 157)
(35, 157)


In [5]:
# empty_rows.to_csv('재무비율null.csv',index=False, encoding='UTF-8-sig')

In [6]:
# 결측치 기업들 feature 컬럼 제외
df = empty_rows.iloc[:, :9]
df['label'] = empty_rows['label']

df.reset_index(drop=True, inplace=True)
df.head()

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,label
0,(주)대호에이엘,69460,2014,1,24,32402,1차 금속 제조업,2002-11-11,NaN,1
1,(주)신화다이나믹스,1770,2012,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1
2,(주)신화다이나믹스,1770,2013,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1
3,(주)신화다이나믹스,1770,2014,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1
4,(주)신화다이나믹스,1770,2015,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1


### 결측치 기업 개별재무비율로 대체

In [7]:
df['회계년도'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 89 entries, 0 to 88
Series name: 회계년도
Non-Null Count  Dtype
--------------  -----
89 non-null     int64
dtypes: int64(1)
memory usage: 840.0 bytes


In [8]:
listed['회계년도'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 28142 entries, 0 to 28141
Series name: 회계년도
Non-Null Count  Dtype 
--------------  ----- 
28142 non-null  object
dtypes: object(1)
memory usage: 220.0+ KB


In [9]:
listed['회계년도'] = listed['회계년도'].astype(str).str[:4].astype(int)

In [10]:
# check = pd.merge(df, listed, on = ['회사명', '거래소코드', '회계년도', '소속코드', '통계청 한국표준산업분류 코드 10차(대분류)', '산업코드',
       # '산업명', '상장일', '상장폐지일'], how = 'left' )

check = pd.merge(df, listed, on = ['거래소코드', '회계년도'], how = 'left')

In [11]:
check.columns

Index(['회사명_x', '거래소코드', '회계년도', '소속코드_x', '통계청 한국표준산업분류 코드 10차(대분류)_x',
       '산업코드_x', '산업명_x', '상장일_x', '상장폐지일_x', 'label',
       ...
       '세금과공과(IFRS)(백만원)', '감가상각비(IFRS)(백만원)', '종업원1인당 부가가치(IFRS)(백만원).1',
       '총자본투자효율(IFRS).1', '기계투자효율(IFRS).1', '부가가치율(IFRS).1', '종업원수(IFRS)',
       '매출채권 대 상,제품비율(IFRS)', '상품,제품회전률(IFRS)', '원,부재료회전률(IFRS)'],
      dtype='object', length=167)

In [12]:
check.drop(columns = ['회사명_y', '소속코드_y', '통계청 한국표준산업분류 코드 10차(대분류)_y',
       '산업코드_y', '산업명_y', '상장일_y', '상장폐지일_y'], inplace = True)

check.rename(columns={'회사명_x' : '회사명',
                      '소속코드_x' : '소속코드',
                      '통계청 한국표준산업분류 코드 10차(대분류)_x' : '통계청 한국표준산업분류 코드 10차(대분류)',
                      '산업코드_x' : '산업코드',
                      '산업명_x' : '산업명',
                      '상장일_x' : '상장일',
                      '상장폐지일_x' : '상장폐지일'}, inplace = True)

# label 컬럼을 분리
label_column = check.pop('label')

# 분리한 label 컬럼을 맨 마지막에 추가
check['label'] = label_column

In [13]:
# check.to_csv('재무비율_개별로대체.csv',index=False,encoding='UTF-8-sig')

In [14]:
check.columns[check.isnull().sum() > 0]

Index(['상장폐지일', '매출채권 대 상', '상품', '원', '매출채권 대 상,제품비율(IFRS)', '상품,제품회전률(IFRS)',
       '원,부재료회전률(IFRS)'],
      dtype='object')

In [15]:
check.columns[:157] == kospi_ratio.columns[:159]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [16]:
def clean_column_name(col_name):
    # col_name = col_name.replace('(*)', '').replace('(원)', '').replace('(백만원)', '')
    # col_name = col_name.replace('[제조]', '')
    # col_name = col_name.replace('(천원)', '')
    col_name = col_name.replace('(IFRS연결)', '(IFRS)')
    # col_name = col_name.replace('.1', '')

    return col_name.strip()  


kospi_ratio.columns = [clean_column_name(col) for col in kospi_ratio.columns]
kospi_ratio

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,총자본증가율(IFRS),...,금융비용(IFRS)(백만원),임차료(IFRS)(백만원),세금과공과(IFRS)(백만원),감가상각비(IFRS)(백만원),종업원1인당 부가가치(IFRS)(백만원).1,총자본투자효율(IFRS).1,기계투자효율(IFRS).1,부가가치율(IFRS).1,종업원수(IFRS),label
0,(주)다이나믹디자인,145210,2015,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-4.56,...,3589.40,0.0,0.0,14203.75,NaN,19.72,98.91,29.06,NaN,1
1,(주)다이나믹디자인,145210,2016,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-12.91,...,0.00,0.0,0.0,0.00,NaN,-3.89,0.00,-5.57,NaN,1
2,(주)다이나믹디자인,145210,2017,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-17.07,...,3990.88,0.0,0.0,11694.91,NaN,8.59,44.66,12.60,NaN,1
3,(주)대우건설,47040,2012,1,41,64101,종합 건설업,2001-03-23,NaN,4.13,...,94086.89,124047.0,54145.0,51212.00,NaN,13.73,2422.60,16.47,NaN,0
4,(주)대우건설,47040,2013,1,41,64101,종합 건설업,2001-03-23,NaN,2.67,...,86015.38,123239.0,62921.0,59003.00,NaN,4.00,731.03,4.62,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,한솔아이원스(주),114810,2017,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
388,한솔아이원스(주),114810,2018,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
389,한솔아이원스(주),114810,2019,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,0.00,...,3425.26,0.0,0.0,8607.60,NaN,20.66,165.10,32.15,NaN,1
390,한솔아이원스(주),114810,2020,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,-12.56,...,2408.62,0.0,0.0,7715.50,NaN,27.08,192.48,30.81,NaN,1


In [17]:
check.columns[:157] == kospi_ratio.columns[:157]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,

In [18]:
# A와 B 데이터프레임이 있다고 가정
kospi_ratio_columns = kospi_ratio.columns.tolist()  # A 데이터프레임의 컬럼 이름 리스트
check_columns = check.columns.tolist()  # B 데이터프레임의 컬럼 이름 리스트

# 빈 데이터프레임 생성
df_comparison = pd.DataFrame(columns=['kospi_ratio_columns', 'check_columns'])

# 빈 데이터프레임에 A와 B의 컬럼 값을 넣기
df_comparison['kospi_ratio_columns'] = pd.Series(kospi_ratio_columns)
df_comparison['check_columns'] = pd.Series(check_columns)

# 결과 출력
df_comparison

,kospi_ratio_columns,check_columns
0,회사명,회사명
1,거래소코드,거래소코드
2,회계년도,회계년도
3,소속코드,소속코드
4,통계청 한국표준산업분류 코드 10차(대분류),통계청 한국표준산업분류 코드 10차(대분류)
...,...,...
152,총자본투자효율(IFRS).1,총자본투자효율(IFRS).1
153,기계투자효율(IFRS).1,기계투자효율(IFRS).1
154,부가가치율(IFRS).1,부가가치율(IFRS).1
155,종업원수(IFRS),종업원수(IFRS)


- 85 매출채권 대 상,제품비율(IFRS) vs 매출채권 대 상,
- 118 상품,제품회전률(IFRS) vs 상품
- 119 원,부재료회전률(IFRS) vs 원
- 156 label vs 매출채권 대 상,제품비율(IFRS)
- 157 X vs 상품,제품회전률(IFRS)
- 158 X vs 원,부재료회전률(IFRS)


개별재무비율로 채운 데이터셋 
- '매출채권 대 상,' 컬럼과 '매출채권 대 상,제품비율(IFRS)'에서 같은 값을 기업에 따라 각각 제출한듯
- '상품' 컬럼과 '상품,제품회전률(IFRS)'에서 같은 값을 기업에 따라 각각 제출한듯
- '원' 컬럼과 '원,부재료회전률(IFRS)'에서 같은 값을 기업에 따라 각각 제출한듯

In [19]:
len(kospi_ratio.columns)

157

In [20]:
len(check.columns)

160

In [21]:
# '매출채권 대 상'과 '매출채권 대 상,제품비율(IFRS)' 컬럼을 합쳐서 '매출채권 대 상,제품비율(IFRS)'로 업데이트

check['매출채권 대 상'] = check['매출채권 대 상'].combine_first(check['매출채권 대 상,제품비율(IFRS)'])

# 중복된 뒤에 컬럼은 삭제
check.drop(columns=['매출채권 대 상,제품비율(IFRS)'], inplace = True)

# 이름은 다시 제대로 된 이름으로 수정
check.rename(columns={'매출채권 대 상' : '매출채권 대 상,제품비율(IFRS)'}, inplace = True)
check['매출채권 대 상,제품비율(IFRS)'].isnull().sum()

0

In [22]:
len(check.columns)

159

In [23]:
# 남은 2개도 똑같이 처리

check['상품'] = check['상품'].combine_first(check['상품,제품회전률(IFRS)'])
check['원'] = check['원'].combine_first(check['원,부재료회전률(IFRS)'])

# 중복된 뒤에 컬럼은 삭제
check.drop(columns=['상품,제품회전률(IFRS)', '원,부재료회전률(IFRS)'], inplace = True)

# 이름은 다시 제대로 된 이름으로 수정
check.rename(columns={'상품' : '상품,제품회전률(IFRS)', '원' : '원,부재료회전률(IFRS)'}, inplace = True)
print(check['상품,제품회전률(IFRS)'].isnull().sum())
print(check['원,부재료회전률(IFRS)'].isnull().sum())

0
0


In [24]:
len(check.columns)

157

In [25]:
check.isnull().sum()

회사명                         0
거래소코드                       0
회계년도                        0
소속코드                        0
통계청 한국표준산업분류 코드 10차(대분류)    0
                           ..
총자본투자효율(IFRS).1             0
기계투자효율(IFRS).1              0
부가가치율(IFRS).1               0
종업원수(IFRS)                  0
label                       0
Length: 157, dtype: int64

In [26]:
# A와 B 데이터프레임이 있다고 가정
kospi_ratio_columns = kospi_ratio.columns.tolist()  # A 데이터프레임의 컬럼 이름 리스트
check_columns = check.columns.tolist()  # B 데이터프레임의 컬럼 이름 리스트

# 빈 데이터프레임 생성
df_comparison = pd.DataFrame(columns=['kospi_ratio_columns', 'check_columns'])

# 빈 데이터프레임에 A와 B의 컬럼 값을 넣기
df_comparison['kospi_ratio_columns'] = pd.Series(kospi_ratio_columns)
df_comparison['check_columns'] = pd.Series(check_columns)

# 결과 출력
df_comparison

,kospi_ratio_columns,check_columns
0,회사명,회사명
1,거래소코드,거래소코드
2,회계년도,회계년도
3,소속코드,소속코드
4,통계청 한국표준산업분류 코드 10차(대분류),통계청 한국표준산업분류 코드 10차(대분류)
...,...,...
152,총자본투자효율(IFRS).1,총자본투자효율(IFRS).1
153,기계투자효율(IFRS).1,기계투자효율(IFRS).1
154,부가가치율(IFRS).1,부가가치율(IFRS).1
155,종업원수(IFRS),종업원수(IFRS)


kospi_ratio에서 개별로 채운 기업들 삭제 후 concat

In [29]:
# 값이 없는 행 삭제
kospi_ratio_cleaned = kospi_ratio.drop(empty_rows.index)
kospi_ratio_cleaned

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,총자본증가율(IFRS),...,금융비용(IFRS)(백만원),임차료(IFRS)(백만원),세금과공과(IFRS)(백만원),감가상각비(IFRS)(백만원),종업원1인당 부가가치(IFRS)(백만원).1,총자본투자효율(IFRS).1,기계투자효율(IFRS).1,부가가치율(IFRS).1,종업원수(IFRS),label
0,(주)다이나믹디자인,145210,2015,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-4.56,...,3589.40,0.00,0.00,14203.75,NaN,19.72,98.91,29.06,NaN,1
1,(주)다이나믹디자인,145210,2016,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-12.91,...,0.00,0.00,0.00,0.00,NaN,-3.89,0.00,-5.57,NaN,1
2,(주)다이나믹디자인,145210,2017,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-17.07,...,3990.88,0.00,0.00,11694.91,NaN,8.59,44.66,12.60,NaN,1
3,(주)대우건설,47040,2012,1,41,64101,종합 건설업,2001-03-23,NaN,4.13,...,94086.89,124047.00,54145.00,51212.00,NaN,13.73,2422.60,16.47,NaN,0
4,(주)대우건설,47040,2013,1,41,64101,종합 건설업,2001-03-23,NaN,2.67,...,86015.38,123239.00,62921.00,59003.00,NaN,4.00,731.03,4.62,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,참존글로벌(주),158310,2018,4,29,32902,기타 기계 및 장비 제조업,2013-04-03,2022/11/09,36.55,...,846.49,0.00,0.00,316.32,NaN,-13.80,-16451.76,-125.42,NaN,1
382,퀀타피아(주),78940,2018,6,46,74604,도매 및 상품 중개업,2004-08-27,NaN,-20.54,...,1364.41,119.33,221.42,1458.23,NaN,-22.07,-2398.22,-42.05,NaN,1
389,한솔아이원스(주),114810,2019,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,0.00,...,3425.26,0.00,0.00,8607.60,NaN,20.66,165.10,32.15,NaN,1
390,한솔아이원스(주),114810,2020,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,-12.56,...,2408.62,0.00,0.00,7715.50,NaN,27.08,192.48,30.81,NaN,1


In [30]:
# kospi_ratio_cleaned와 check를 위아래로 연결
combined_df = pd.concat([kospi_ratio_cleaned, check], ignore_index=True)
combined_df

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,총자본증가율(IFRS),...,금융비용(IFRS)(백만원),임차료(IFRS)(백만원),세금과공과(IFRS)(백만원),감가상각비(IFRS)(백만원),종업원1인당 부가가치(IFRS)(백만원).1,총자본투자효율(IFRS).1,기계투자효율(IFRS).1,부가가치율(IFRS).1,종업원수(IFRS),label
0,(주)다이나믹디자인,145210,2015,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-4.56,...,3589.40,0.0,0.0,14203.75,NaN,19.72,98.91,29.06,NaN,1
1,(주)다이나믹디자인,145210,2016,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-12.91,...,0.00,0.0,0.0,0.00,NaN,-3.89,0.00,-5.57,NaN,1
2,(주)다이나믹디자인,145210,2017,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,-17.07,...,3990.88,0.0,0.0,11694.91,NaN,8.59,44.66,12.60,NaN,1
3,(주)대우건설,47040,2012,1,41,64101,종합 건설업,2001-03-23,NaN,4.13,...,94086.89,124047.0,54145.0,51212.00,NaN,13.73,2422.60,16.47,NaN,0
4,(주)대우건설,47040,2013,1,41,64101,종합 건설업,2001-03-23,NaN,2.67,...,86015.38,123239.0,62921.0,59003.00,NaN,4.00,731.03,4.62,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,한솔아이원스(주),114810,2014,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,-0.08,...,1153.62,0.0,0.0,5874.10,80.59,24.18,64.97,36.25,311.0,1
388,한솔아이원스(주),114810,2015,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,26.94,...,986.84,0.0,0.0,4275.95,64.50,17.79,126.87,33.75,363.0,1
389,한솔아이원스(주),114810,2016,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,19.93,...,3567.48,0.0,0.0,6968.28,31.26,7.78,47.27,14.45,393.0,1
390,한솔아이원스(주),114810,2017,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,18.82,...,5048.10,0.0,0.0,7923.59,61.45,17.01,121.67,24.62,519.0,1


In [35]:
combined_df.sort_values(by=['거래소코드', '회계년도'], ascending=[True, True])

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,총자본증가율(IFRS),...,금융비용(IFRS)(백만원),임차료(IFRS)(백만원),세금과공과(IFRS)(백만원),감가상각비(IFRS)(백만원),종업원1인당 부가가치(IFRS)(백만원).1,총자본투자효율(IFRS).1,기계투자효율(IFRS).1,부가가치율(IFRS).1,종업원수(IFRS),label
101,현대건설(주),720,2013,1,41,64102,종합 건설업,1984-12-22,NaN,15.06,...,157.00,0.00,0.00,93263.00,NaN,13.79,389.30,14.60,NaN,1
102,현대건설(주),720,2014,1,41,64102,종합 건설업,1984-12-22,NaN,24.40,...,31992.00,0.00,0.00,154054.00,NaN,13.15,414.72,13.89,NaN,1
103,현대건설(주),720,2015,1,41,64102,종합 건설업,1984-12-22,NaN,6.06,...,29408.00,0.00,0.00,195819.00,NaN,14.03,452.89,14.12,NaN,1
104,현대건설(주),720,2016,1,41,64102,종합 건설업,1984-12-22,NaN,2.70,...,6913.00,0.00,0.00,190461.00,NaN,13.96,412.52,14.74,NaN,1
311,백광산업(주),1340,2015,1,20,32001,화학물질 및 화학제품 제조업; 의약품 제외,1976-06-10,NaN,-31.06,...,3179.30,955.06,0.00,19990.25,204.45,7.92,52.38,18.99,120.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,(주)씨엠에스에듀,225330,2016,4,85,168505,교육 서비스업,2016-04-07,2022/03/17,123.95,...,-77.89,4832.26,346.33,1773.32,NaN,34.09,540.44,39.08,NaN,1
168,(주)씨엠에스에듀,225330,2017,4,85,168505,교육 서비스업,2016-04-07,2022/03/17,9.96,...,-346.00,5965.99,493.86,2982.47,NaN,39.66,419.82,43.79,NaN,1
347,(주)에스에스알,275630,2018,5,58,105802,출판업,2018-08-06,NaN,0.00,...,-43.93,0.00,0.00,693.64,60.95,23.80,0.00,51.01,101.0,1
348,(주)에스에스알,275630,2019,5,58,105802,출판업,2018-08-06,NaN,2.29,...,-227.10,0.00,0.00,858.04,61.66,24.93,0.00,50.93,107.0,1


---
### 전체에 대해서 값이 없는 컬럼 삭제

In [38]:
# 값이 아예 없는(전체 결측치인) 컬럼 찾기
empty_columns = combined_df.columns[combined_df.isnull().all()]
print(empty_columns)
print(len(empty_columns))

Index([], dtype='object')
0


In [39]:
# 값이 아예 없는(전체 결측치인) 컬럼 찾기
empty_columns = kospi_ratio.columns[kospi_ratio.isnull().all()]
print(empty_columns)
print(len(empty_columns))

Index(['종업원1인당 부가가치증가율(IFRS)', '종업원수증가율(IFRS)', '종업원1인당 매출액증가율(IFRS)',
       '종업원1인당 인건비증가율(IFRS)', '사내유보율(IFRS)', '사내유보 대 자기자본비율(IFRS)',
       '평균배당률(IFRS)', '자기자본배당률(IFRS)', '배당성향(IFRS)', '1주당순자산(IFRS)(원)',
       '종업원1인당 부가가치(IFRS)(백만원)', '종업원1인당 매출액(IFRS)(백만원)',
       '종업원1인당 정상영업이익(IFRS)(백만원)', '종업원1인당 순이익(IFRS)(백만원)',
       '종업원1인당 인건비(IFRS)(백만원)', '노동장비율(IFRS)', '기계장비율(IFRS)', '자본집약도(IFRS)',
       '종업원1인당 부가가치(IFRS)(백만원).1', '종업원수(IFRS)'],
      dtype='object')
20


In [ ]:
kospi_ratio.drop(columns = empty_columns, inplace=True)

개별로 채우니까 연결에서 전체 값이 없던 컬럼들 값이 채워짐

In [40]:
# combined_df.to_csv('../data/processed_data/재무비율_처리완.csv', index=False, encoding='UTF-8-sig')